In [1]:
import Augmentor
from PIL import Image
from tqdm import tqdm
import os
import re
import shutil

image_dir = './input/images'
mask_dir = './input/masks'
output_dir = './output'

p = Augmentor.Pipeline(image_dir)
# Point to a directory containing ground truth data.
# Images with the same file names will be added as ground truth data
# and augmented in parallel to the original data.
p.ground_truth(mask_dir)

p.rotate_without_crop(probability=1, max_left_rotation=120, max_right_rotation=120)
p.flip_left_right(probability=1)
p.flip_top_bottom(probability=1)
# p.skew(probability=1)
# p.random_distortion(probability=1, grid_width=6, grid_height=6, magnitude=8)
p.sample(10000)


Executing Pipeline:   0%|          | 0/10000 [00:00<?, ? Samples/s]

Initialised with 64 image(s) found.
Output directory set to ./input/images/output.64 ground truth image(s) found.


Processing <PIL.Image.Image image mode=RGBA size=1294x800 at 0x7F06B96AAC88>:  86%|████████▌ | 8579/10000 [05:38<00:01, 1143.15 Samples/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

Processing <PIL.Image.Image image mode=RGBA size=1416x312 at 0x7F06B94CA940>: 100%|██████████| 10000/10000 [05:40<00:00, 29.33 Samples/s] 


In [2]:
# Augmentor genenrates images into ./images/output in following formats:
# - original image: images_original_rock-8.png_325b4fc9-cf0b-4c6a-9574-dc9cc13da6b3.png
# - masked image  : _groundtruth_(1)_images_rock-8.png_325b4fc9-cf0b-4c6a-9574-dc9cc13da6b3.png
# Now, we need to convert them into following structure:
# - original image: images/rock-8.png_325b4fc9-cf0b-4c6a-9574-dc9cc13da6b3.png
# - masked image  : masks/rock-8.png_325b4fc9-cf0b-4c6a-9574-dc9cc13da6b3.png

target_root_dir = os.path.join(output_dir, 'stage1_train')
os.mkdir(target_root_dir)
    
augmentor_output_dir = os.path.join(image_dir, 'output')
for filename in tqdm(os.listdir(augmentor_output_dir)):
    if re.search(r"^_groundtruth_", filename):
        new_filename = re.sub(
            r"^_groundtruth_\(1\)_images_",
            '',
            filename
        )
        image_or_mask_subdir = 'masks'
    else:
        new_filename = re.sub(
            r"^images_original_",
            '',
            filename
        )
        image_or_mask_subdir = 'images'
        
    image_full_path = os.path.join(augmentor_output_dir, filename)
    target_file_root_dir = os.path.join(target_root_dir, new_filename)
    if not os.path.exists(target_file_root_dir):
        os.mkdir(target_file_root_dir)

    target_dir = os.path.join(target_file_root_dir, image_or_mask_subdir)
    if not os.path.exists(target_dir):
        os.mkdir(target_dir)
    
    shutil.move(
        image_full_path,
        os.path.join(target_dir, new_filename)
    )
    
    
    


100%|██████████| 20000/20000 [00:01<00:00, 10811.88it/s]


In [3]:

for filename in tqdm(os.listdir(image_dir)):
    if os.path.isdir(filename):
        continue
        
    image_full_path = os.path.join(image_dir, filename)

    target_file_root_dir = os.path.join(target_root_dir, filename)
    if not os.path.exists(target_file_root_dir):
        os.mkdir(target_file_root_dir)

    target_dir = os.path.join(target_file_root_dir, 'images')
    if not os.path.exists(target_dir):
        os.mkdir(target_dir)
    
    shutil.copy(
        image_full_path,
        os.path.join(target_dir, filename)
    )
    
    
    


100%|██████████| 65/65 [00:00<00:00, 426.35it/s]


In [4]:

for filename in tqdm(os.listdir(mask_dir)):
    if os.path.isdir(filename):
        continue
        
    image_full_path = os.path.join(mask_dir, filename)

    target_file_root_dir = os.path.join(target_root_dir, filename)
    if not os.path.exists(target_file_root_dir):
        os.mkdir(target_file_root_dir)

    target_dir = os.path.join(target_file_root_dir, 'masks')
    if not os.path.exists(target_dir):
        os.mkdir(target_dir)
    
    shutil.copy(
        image_full_path,
        os.path.join(target_dir, filename)
    )
    


100%|██████████| 64/64 [00:00<00:00, 4088.33it/s]
